## Dati:
#### Coordinate
IT_NORD
Milano: 45.44660816101568, 9.241619983380145
Bologna:44.47832978671284, 11.46982778693447

media tra le città più grandi
44.962468974, 10.354948926

In [10]:
#!pip install pvlib

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pyarrow 
import pvlib
from pvlib.location import Location
from scipy.interpolate import interp1d
from scipy.integrate import quad

In [8]:
filename = "/home/samu/LCP_B_project/lcpb_files/ssrd_solar.zst"
years = ['22','23','24'] 
zones = ['IT_NORD']


df_sol = pd.read_parquet(filename) # this reads .zst files efficiently
df_sol = df_sol[df_sol['node_id'].isin(zones)]
df_sol.index = pd.to_datetime(df_sol.index)  # Convert index to DatetimeIndex
#df_sol['year'] = df_sol.index.year
#df_sol['month'] = df_sol.index.month
#df_sol['day'] = df_sol.index.day
#df_sol['hour'] = df_sol.index.hour
#df_sol['date'] = df_sol.index.date
df_sol

,node_id,ssrd
date_time,,
2021-12-31 18:00:00+00:00,IT_NORD,0.000000
2022-01-01 06:00:00+00:00,IT_NORD,516877.928058
2022-01-01 18:00:00+00:00,IT_NORD,0.000000
2022-01-02 06:00:00+00:00,IT_NORD,445703.597122
2022-01-02 18:00:00+00:00,IT_NORD,0.000000
...,...,...
2025-01-23 06:00:00+00:00,IT_NORD,236703.424460
2025-01-23 18:00:00+00:00,IT_NORD,0.000000
2025-01-24 06:00:00+00:00,IT_NORD,585959.712230


In [11]:
latitude = 44.962468974
longitude = 10.354948926
timezone = 'Europe/Rome'
site = Location(latitude, longitude, tz=timezone)

In [12]:
def process_row(row):
    start_utc = row.name
    start = start_utc.tz_convert(timezone)
    E_cum = row['ssrd']
    hour = start.hour
    
    end = start + pd.Timedelta(hours=12)
    times = pd.date_range(start, end, freq='1h', tz=timezone)
    solpos = site.get_solarposition(times) 
    
    theta_z = np.radians(solpos['zenith']) ##zenith angles each of the 12 hours
    cos_theta = np.clip(np.cos(theta_z), 0, None)
    
    integral = np.sum(cos_theta)
    
    base_result = { ### schema for the final dataframe
        'start_time': start,
        'end_time': end,
        'node_id': row['node_id'],
        'original_datetime': start_utc
    }
    
    if integral == 0:
        # night
        results = []
        for t0 in range(6 if hour == 6 else 18, 18 if hour == 6 else 30, 3):
            t1 = t0 + 3
            results.append({
                **base_result,
                'start_time': start + pd.Timedelta(hours=t0-hour),
                'end_time': start + pd.Timedelta(hours=t1-hour),
                'E_3h': 0.0  # Set to zero the 3hours timeframes with no radiations
            })
        return pd.DataFrame(results)
    
    I0 = E_cum / (integral)
    I_t = I0 * cos_theta
    #print(I_t) #I(t) calculated every hour (try printing to check)
    t_decimal = np.array([(t.hour + t.minute/60) + (t.day-start.day)*24 for t in times])
    
    #if len(t_decimal) < 4 or np.all(I_t == 0): ##no need no more
    #    return pd.DataFrame([{**base_result, 'E_3h': 0.0}])
    
    I_func = interp1d(t_decimal, I_t, kind='linear', bounds_error=False, fill_value=0)
    
    results = []
    for t0 in range(int(t_decimal[0]), int(t_decimal[-1]), 3):
        t1 = t0 + 3
        
        if t0 >= t_decimal[-1] or t1 <= t_decimal[0]:
            continue
            
        try:
            E_i, _ = quad(I_func, t0, t1)
            results.append({
                **base_result,
                'start_time': start + pd.Timedelta(hours=t0-hour),
                'end_time': start + pd.Timedelta(hours=t1-hour),
                'E_3h': E_i 
            })
        except Exception as e:
            print(f"Integration error at {start_utc}: {e}")
            results.append({
                **base_result, ## set to zero the change of hour
                'start_time': start + pd.Timedelta(hours=t0-hour),
                'end_time': start + pd.Timedelta(hours=t1-hour),
                'E_3h': 0.0  
            })
    
    return pd.DataFrame(results) if results else pd.DataFrame([{**base_result, 'E_3h': 0.0}])
        

def process_dataframe(df_sol):
    all_blocks = []
    
    for idx, row in df_sol.iterrows():
        try:
            result = process_row(row)
            if result is not None:
                all_blocks.append(result)
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
    
    if not all_blocks:
        return pd.DataFrame()
        
    return pd.concat(all_blocks, ignore_index=True)

df_result = process_dataframe(df_sol)
df_result.head(50)


,start_time,end_time,node_id,original_datetime,E_3h
0,2021-12-31 18:00:00+01:00,2021-12-31 21:00:00+01:00,IT_NORD,2021-12-31 18:00:00+00:00,0.000000
1,2021-12-31 21:00:00+01:00,2022-01-01 00:00:00+01:00,IT_NORD,2021-12-31 18:00:00+00:00,0.000000
2,2022-01-01 00:00:00+01:00,2022-01-01 03:00:00+01:00,IT_NORD,2021-12-31 18:00:00+00:00,0.000000
3,2022-01-01 03:00:00+01:00,2022-01-01 06:00:00+01:00,IT_NORD,2021-12-31 18:00:00+00:00,0.000000
4,2022-01-01 07:00:00+01:00,2022-01-01 10:00:00+01:00,IT_NORD,2022-01-01 06:00:00+00:00,64894.410882
5,2022-01-01 10:00:00+01:00,2022-01-01 13:00:00+01:00,IT_NORD,2022-01-01 06:00:00+00:00,248799.646036
6,2022-01-01 13:00:00+01:00,2022-01-01 16:00:00+01:00,IT_NORD,2022-01-01 06:00:00+00:00,190592.097227
7,2022-01-01 16:00:00+01:00,2022-01-01 19:00:00+01:00,IT_NORD,2022-01-01 06:00:00+00:00,12591.773912
8,2022-01-01 18:00:00+01:00,2022-01-01 21:00:00+01:00,IT_NORD,2022-01-01 18:00:00+00:00,0.000000
9,2022-01-01 21:00:00+01:00,2022-01-02 00:00:00+01:00,IT_NORD,2022-01-01 18:00:00+00:00,0.000000
